In [53]:
# 📦 Imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score


In [54]:
# ⚙️ Config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"  # For testing purposes, use CPU
BATCH_SIZE = 32
EPOCHS = 50
HIDDEN_DIM = 32

In [55]:
device 

device(type='cuda')

In [56]:
# 📄 Load Data
df = pd.read_csv("data/allHorizonData_cut.csv")

# 🧹 Build Sequences and Labels
seq_inputs, static_inputs, labels, horizons = [], [], [], []

for _, row in df.iterrows():
    rewards = [row[f"r{i}"] for i in range(1, 5)]
    choices = [row[f"c{i}"] for i in range(1, 5)]
    seq = list(zip(rewards, choices))  # [4, 2]

    static = [row["gameLength"], row["uc"]]
    label = row["c5"]-1

    seq_inputs.append(seq)
    static_inputs.append(static)
    labels.append(label)
    horizons.append(row["gameLength"])

# Convert to torch tensors
seq_tensor = torch.tensor(seq_inputs).float()
static_tensor = torch.tensor(static_inputs).float()
label_tensor = torch.tensor(labels).float()
horizon_tensor = torch.tensor(horizons).int()


In [57]:
# 🧪 Train/Test Split
dataset = TensorDataset(seq_tensor, static_tensor, label_tensor, horizon_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [58]:

# 🧠 LSTM Model
class HorizonLSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=HIDDEN_DIM):
        super(HorizonLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim + 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, seq_input, static_input):
        lstm_out, _ = self.lstm(seq_input)
        last_hidden = lstm_out[:, -1, :]
        combined = torch.cat((last_hidden, static_input), dim=1)
        return self.fc(combined)

In [59]:

# 🏋️ Train
model = HorizonLSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

for epoch in range(EPOCHS):
    model.train()
    epoch_preds = []
    epoch_labels = []
    total_loss = 0

    for seq_batch, static_batch, label_batch, _ in train_loader:
        seq_batch = seq_batch.to(device)
        static_batch = static_batch.to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()
        outputs = model(seq_batch, static_batch).squeeze()
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 👇 Convert predictions to 0 or 1
        preds = (outputs > 0.5).int()
        epoch_preds.extend(preds.cpu().numpy())
        epoch_labels.extend(label_batch.cpu().numpy())

    if epoch % 5 == 0:
        correct = np.sum(np.array(epoch_preds) == np.array(epoch_labels))
        acc = correct / len(epoch_preds)
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch}: Loss = {avg_loss:.4f}, Accuracy = {acc:.4f}")



Epoch 0: Loss = 0.6932
Epoch 5: Loss = 0.4433
Epoch 10: Loss = 0.3519
Epoch 15: Loss = 0.3207
Epoch 20: Loss = 0.4849
Epoch 25: Loss = 0.3995
Epoch 30: Loss = 0.3727
Epoch 35: Loss = 0.4716
Epoch 40: Loss = 0.4535
Epoch 45: Loss = 0.4748


In [63]:

# 📈 Evaluate
model.eval()
all_preds, all_labels, all_horizons = [], [], []

with torch.no_grad():
    for seq_batch, static_batch, label_batch, horizon_batch in val_loader:
        seq_batch = seq_batch.to(device)
        static_batch = static_batch.to(device)

        preds = model(seq_batch, static_batch).squeeze().cpu().numpy()
        bin_preds = (preds > 0.5).astype(int)

        all_preds.extend(bin_preds)
        all_labels.extend(label_batch.numpy())
        all_horizons.extend(horizon_batch.numpy())


In [64]:

# 🎯 Overall Accuracy
overall_acc = accuracy_score(all_labels, all_preds)
print(f"\nOverall Accuracy: {overall_acc:.4f}")



Overall Accuracy: 0.7719


In [65]:

# 📊 Horizon-wise Accuracy
h1_acc = accuracy_score(
    [l for l, h in zip(all_labels, all_horizons) if h == 5],
    [p for p, h in zip(all_preds, all_horizons) if h == 5]
)
h6_acc = accuracy_score(
    [l for l, h in zip(all_labels, all_horizons) if h == 10],
    [p for p, h in zip(all_preds, all_horizons) if h == 10]
)

print(f"Horizon 1 Accuracy: {h1_acc:.4f}")
print(f"Horizon 6 Accuracy: {h6_acc:.4f}")

Horizon 1 Accuracy: 0.8264
Horizon 6 Accuracy: 0.7193
